In [12]:
import pandas as pd
import numpy as np
import requests
import reverse_geocoder as rg
#from functions import create_category_folders_from_csv

In [13]:
data = pd.read_csv("../photos_final.csv")

In [14]:
#add index column
data.reset_index(inplace=True)
data

,level_0,index,image_name,path,date_time,latitude,longitude,cluster,categories,folder_path
0,0,0,20240902_150137.jpg,..\photos_final\20240902_150137.jpg,2024:09:02 15:01:38,NaN,NaN,cluster_0,2024_09_02_Randonnée,2024/Automne/2024_09_02_Randonnée
1,1,1,20240902_152654.jpg,..\photos_final\20240902_152654.jpg,2024:09:02 15:26:54,NaN,NaN,cluster_0,2024_09_02_Randonnée,2024/Automne/2024_09_02_Randonnée
2,2,2,20240902_154603.jpg,..\photos_final\20240902_154603.jpg,2024:09:02 15:46:03,NaN,NaN,cluster_0,2024_09_02_Randonnée,2024/Automne/2024_09_02_Randonnée
3,3,3,20240902_154610.jpg,..\photos_final\20240902_154610.jpg,2024:09:02 15:46:10,NaN,NaN,cluster_0,2024_09_02_Randonnée,2024/Automne/2024_09_02_Randonnée
4,4,4,20240902_154612.jpg,..\photos_final\20240902_154612.jpg,2024:09:02 15:46:13,NaN,NaN,cluster_1,Autres/Randonnée,2024/Automne/Autres/Randonnée
...,...,...,...,...,...,...,...,...,...,...
93,93,93,IMG_20250309_151441.jpg,..\photos_final\IMG_20250309_151441.jpg,2025:03:09 15:14:42,48.408989,-71.053697,cluster_23,2025_03_09_Nourriture,2025/Printemps/Saguenay_CA/2025_03_09_Nourriture
94,94,94,IMG_20250309_200311.jpg,..\photos_final\IMG_20250309_200311.jpg,2025:03:09 20:03:11,NaN,NaN,cluster_24,Autres/Bâtiment,2025/Printemps/Autres/Bâtiment
95,95,95,IMG_20250309_200336.jpg,..\photos_final\IMG_20250309_200336.jpg,2025:03:09 20:03:37,48.425122,-71.067358,cluster_25,2025_03_09_Nourriture,2025/Printemps/Saguenay_CA/2025_03_09_Nourriture
96,96,96,IMG_20250309_200346.jpg,..\photos_final\IMG_20250309_200346.jpg,2025:03:09 20:03:46,48.425122,-71.067358,cluster_25,2025_03_09_Nourriture,2025/Printemps/Saguenay_CA/2025_03_09_Nourriture


In [15]:
def get_season(month):
    if month in [12, 1, 2]:
        return "Hiver"
    elif month in [3, 4, 5]:
        return "Printemps"
    elif month in [6, 7, 8]:
        return "Été"
    elif month in [9, 10, 11]:
        return "Automne"
    else:
        return None

In [16]:
def reverse_geocode_lib(lat, lon):
    try:
        result = rg.search((lat, lon))[0]
        return f"{result['name']}_{result['cc']}"
    except Exception as e:
        print(f"Erreur sur coords ({lat}, {lon}) : {e}")
        return None

In [17]:
def get_localisation(latitude, longitude, cache):
    lat = round(latitude, 3)
    lon = round(longitude, 3)
    coords = (lat, lon)

    if coords in cache:
        localisation = cache[coords]
    else:
        localisation = reverse_geocode_lib(*coords)
        cache[coords] = localisation

    return localisation

In [18]:
def tree_structure(data):
    tree_paths = []
    cache = {}  # Pour éviter les appels redondants
    for row in data.itertuples():

        date_time = row.date_time.split(":")
        year = int(date_time[0])
        month = int(date_time[1])
        season = get_season(month)

        latitude = row.latitude
        longitude = row.longitude
        localisation = None
        if not np.isnan(latitude) and not np.isnan(longitude):
            localisation = get_localisation(latitude, longitude, cache)

        category = row.categories

        if localisation :
            tree_path = f"{year}/{season}/{localisation}/{category}"
        else:
            tree_path = f"{year}/{season}/{category}"

        tree_paths.append(tree_path)

    data['folder_path'] = tree_paths
    return data

In [19]:
data = tree_structure(data)
data

,level_0,index,image_name,path,date_time,latitude,longitude,cluster,categories,folder_path
0,0,0,20240902_150137.jpg,..\photos_final\20240902_150137.jpg,2024:09:02 15:01:38,NaN,NaN,cluster_0,2024_09_02_Randonnée,2024/Automne/2024_09_02_Randonnée
1,1,1,20240902_152654.jpg,..\photos_final\20240902_152654.jpg,2024:09:02 15:26:54,NaN,NaN,cluster_0,2024_09_02_Randonnée,2024/Automne/2024_09_02_Randonnée
2,2,2,20240902_154603.jpg,..\photos_final\20240902_154603.jpg,2024:09:02 15:46:03,NaN,NaN,cluster_0,2024_09_02_Randonnée,2024/Automne/2024_09_02_Randonnée
3,3,3,20240902_154610.jpg,..\photos_final\20240902_154610.jpg,2024:09:02 15:46:10,NaN,NaN,cluster_0,2024_09_02_Randonnée,2024/Automne/2024_09_02_Randonnée
4,4,4,20240902_154612.jpg,..\photos_final\20240902_154612.jpg,2024:09:02 15:46:13,NaN,NaN,cluster_1,Autres/Randonnée,2024/Automne/Autres/Randonnée
...,...,...,...,...,...,...,...,...,...,...
93,93,93,IMG_20250309_151441.jpg,..\photos_final\IMG_20250309_151441.jpg,2025:03:09 15:14:42,48.408989,-71.053697,cluster_23,2025_03_09_Nourriture,2025/Printemps/Saguenay_CA/2025_03_09_Nourriture
94,94,94,IMG_20250309_200311.jpg,..\photos_final\IMG_20250309_200311.jpg,2025:03:09 20:03:11,NaN,NaN,cluster_24,Autres/Bâtiment,2025/Printemps/Autres/Bâtiment
95,95,95,IMG_20250309_200336.jpg,..\photos_final\IMG_20250309_200336.jpg,2025:03:09 20:03:37,48.425122,-71.067358,cluster_25,2025_03_09_Nourriture,2025/Printemps/Saguenay_CA/2025_03_09_Nourriture
96,96,96,IMG_20250309_200346.jpg,..\photos_final\IMG_20250309_200346.jpg,2025:03:09 20:03:46,48.425122,-71.067358,cluster_25,2025_03_09_Nourriture,2025/Printemps/Saguenay_CA/2025_03_09_Nourriture


In [20]:
data.to_csv("photos_final.csv", index=False)

In [22]:
stop

NameError: name 'stop' is not defined

# Autres

In [4]:
def group_year_images(data):
    group_year = {}

    for row in data.itertuples():
        year = row.date_time.split(":")[0]
        if year not in group_year:
            group_year[year] = []
        group_year[year].append(row.index)

    return group_year

In [5]:
group_year = group_year_images(data)
group_year

{'2024': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48],
 '2025': [49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97]}

In [6]:
for year, image_indexes in group_year.items():
    print(data.iloc[image_indexes][["date_time", "path"]])

              date_time                                 path
0   2024:09:02 15:01:38  ..\photos_final\20240902_150137.jpg
1   2024:09:02 15:26:54  ..\photos_final\20240902_152654.jpg
2   2024:09:02 15:46:03  ..\photos_final\20240902_154603.jpg
3   2024:09:02 15:46:10  ..\photos_final\20240902_154610.jpg
4   2024:09:02 15:46:13  ..\photos_final\20240902_154612.jpg
5   2024:09:02 15:46:24  ..\photos_final\20240902_154624.jpg
6   2024:09:02 15:53:44  ..\photos_final\20240902_155344.jpg
7   2024:09:02 16:16:27  ..\photos_final\20240902_161626.jpg
8   2024:09:21 13:17:58  ..\photos_final\20240921_131758.jpg
9   2024:09:21 13:17:59  ..\photos_final\20240921_131759.jpg
10  2024:09:21 13:55:14  ..\photos_final\20240921_135514.jpg
11  2024:09:21 13:55:27  ..\photos_final\20240921_135526.jpg
12  2024:09:21 13:55:35  ..\photos_final\20240921_135535.jpg
13  2024:09:21 17:50:10  ..\photos_final\20240921_175010.jpg
14  2024:09:21 17:50:14  ..\photos_final\20240921_175013.jpg
15  2024:09:21 17:56:38 

In [7]:
def group_seasons_images(data):
    group_seasons = {}

    for row in data.itertuples():
        date_time = row.date_time.split(":")
        month = int(date_time[1])
        season = ""

        if month in [12, 1, 2]:
            season = "Hiver"
        elif month in [3, 4, 5]:
            season = "Printemps"
        elif month in [6, 7, 8]:
            season = "Été"
        elif month in [9, 10, 11]:
            season = "Automne"

        if season not in group_seasons:
            group_seasons[season] = []
        group_seasons[season].append(row.index)

    return group_seasons

In [8]:
group_seasons = group_seasons_images(data)
for season, image_indexes in group_seasons.items():
    print(f"{season}:")
    print(data.iloc[image_indexes][["date_time", "path"]])
    print()

Automne:
              date_time                                 path
0   2024:09:02 15:01:38  ..\photos_final\20240902_150137.jpg
1   2024:09:02 15:26:54  ..\photos_final\20240902_152654.jpg
2   2024:09:02 15:46:03  ..\photos_final\20240902_154603.jpg
3   2024:09:02 15:46:10  ..\photos_final\20240902_154610.jpg
4   2024:09:02 15:46:13  ..\photos_final\20240902_154612.jpg
5   2024:09:02 15:46:24  ..\photos_final\20240902_154624.jpg
6   2024:09:02 15:53:44  ..\photos_final\20240902_155344.jpg
7   2024:09:02 16:16:27  ..\photos_final\20240902_161626.jpg
8   2024:09:21 13:17:58  ..\photos_final\20240921_131758.jpg
9   2024:09:21 13:17:59  ..\photos_final\20240921_131759.jpg
10  2024:09:21 13:55:14  ..\photos_final\20240921_135514.jpg
11  2024:09:21 13:55:27  ..\photos_final\20240921_135526.jpg
12  2024:09:21 13:55:35  ..\photos_final\20240921_135535.jpg
13  2024:09:21 17:50:10  ..\photos_final\20240921_175010.jpg
14  2024:09:21 17:50:14  ..\photos_final\20240921_175013.jpg
15  2024:09:21 

In [9]:
def reverse_geocode_nominatim(lat, lon):
    url = "https://nominatim.openstreetmap.org/reverse"
    params = {
        'lat': lat,
        'lon': lon,
        'format': 'json',
        'zoom': 10,  # Ville + région
        'addressdetails': 1
    }
    headers = {
        'User-Agent': 'SnapSortBot/1.0'
    }
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        print("Réponse API")
        return response.json().get("address", {})
    else:
        return {}

In [10]:
def group_localisations_images_nominatim(data):
    group_localisations = {}
    cache = {}  # Pour éviter les appels redondants

    for row in data.itertuples():
        localisation = None
        coords = (row.latitude, row.longitude)

        if coords in cache:
            loc = cache[coords]
        else:
            loc = reverse_geocode_nominatim(*coords)
            cache[coords] = loc

        city = loc.get("city") or loc.get("town") or loc.get("village") or None
        country = loc.get("country") or None

        if city and country:
            localisation = f"{city}_{country}"

        if localisation not in group_localisations:
            group_localisations[localisation] = []
        group_localisations[localisation].append(row.index)

    return group_localisations

In [47]:
def group_localisations_images(data):
    group_localisations = {}
    cache = {}  # Pour éviter les appels redondants

    for row in data.itertuples():
        localisation = None
        lat = round(row.latitude, 5)
        lon = round(row.longitude, 5)

        if not np.isnan(lat) and not np.isnan(lon):
            coords = (lat, lon)
            print(f"Traitement des coordonnées : {coords}")
            if coords in cache:
                localisation = cache[coords]
                print("Localisation trouvée dans le cache.")
            else:
                localisation = reverse_geocode_lib(*coords)
                cache[coords] = localisation

        if localisation not in group_localisations:
            group_localisations[localisation] = []
        group_localisations[localisation].append(row.index)

    return group_localisations

In [48]:
group_localisations = group_localisations_images(data)
for localisation, image_indexes in group_localisations.items():
    print(f"{localisation}:")
    print(data.iloc[image_indexes][["date_time", "path"]])
    print()

Traitement des coordonnées : (46.81206, -71.20543)
Traitement des coordonnées : (46.81206, -71.20543)
Traitement des coordonnées : (46.81203, -71.20558)
Traitement des coordonnées : (46.81203, -71.20562)
Traitement des coordonnées : (46.81199, -71.20546)
Traitement des coordonnées : (46.81204, -71.20551)
Traitement des coordonnées : (46.81361, -71.20314)
Traitement des coordonnées : (46.81232, -71.20335)
Traitement des coordonnées : (46.81238, -71.20333)
Traitement des coordonnées : (46.81239, -71.20334)
Traitement des coordonnées : (48.40515, -71.05691)
Traitement des coordonnées : (48.40515, -71.05691)
Traitement des coordonnées : (48.38947, -71.04453)
Traitement des coordonnées : (48.41677, -71.06454)
Traitement des coordonnées : (48.41764, -71.06279)
Traitement des coordonnées : (48.41819, -71.06324)
Traitement des coordonnées : (48.41676, -71.06746)
Traitement des coordonnées : (48.41676, -71.06746)
Traitement des coordonnées : (48.41691, -71.06759)
Traitement des coordonnées : (4

In [49]:
def dictionary_year_seasons_localisation(group_year, group_seasons, group_localisations):
    arborescence = {}

    for year, year_indexes in group_year.items():
        arborescence[year] = {}
        for season, season_indexes in group_seasons.items():

            # Intersection année + saison
            indexes = [idx for idx in year_indexes if idx in season_indexes]

            # Filtrage par localisation
            loc_group = {}
            for idx in indexes:
                localisation_found = False
                for loc, loc_indexes in group_localisations.items():
                    if idx in loc_indexes:
                        loc_group.setdefault(loc, []).append(idx)
                        localisation_found = True
                        break
                if not localisation_found:
                    loc_group.setdefault("Unknown", []).append(idx)

            # Localisations trouvées : structure avec arborescence
            arborescence[year][season] = {
                loc: loc_group[loc] for loc in loc_group
            }

    return arborescence

In [50]:
arborescence = dictionary_year_seasons_localisation(group_year, group_seasons, group_localisations)
arborescence

{'2024': {'Automne': {None: [0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    31,
    32,
    33,
    34,
    35,
    36,
    37,
    38,
    39,
    40,
    41]},
  'Hiver': {None: [42, 43, 44, 45, 46, 47, 48]},
  'Printemps': {}},
 '2025': {'Automne': {},
  'Hiver': {None: [49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 86, 87],
   'Quebec_CA': [63, 64, 65, 66, 67, 68, 69, 70, 71, 72],
   'Saguenay_CA': [73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 88]},
  'Printemps': {None: [60, 61, 62, 94, 97],
   'Saguenay_CA': [89, 90, 91, 92, 93, 95, 96]}}}

In [51]:
def generate_index_paths_dict(tree):
    index_to_path = {}

    for year, seasons in tree.items():
        for season, content in seasons.items():
            for loc, indexes in content.items():
                for idx in indexes:
                    if loc :
                        index_to_path[idx] = f"{year}/{season}/{loc}"
                    else:
                        index_to_path[idx] = f"{year}/{season}"

    return index_to_path

In [52]:
index_to_path = generate_index_paths_dict(arborescence)
index_to_path

{0: '2024/Automne',
 1: '2024/Automne',
 2: '2024/Automne',
 3: '2024/Automne',
 4: '2024/Automne',
 5: '2024/Automne',
 6: '2024/Automne',
 7: '2024/Automne',
 8: '2024/Automne',
 9: '2024/Automne',
 10: '2024/Automne',
 11: '2024/Automne',
 12: '2024/Automne',
 13: '2024/Automne',
 14: '2024/Automne',
 15: '2024/Automne',
 16: '2024/Automne',
 17: '2024/Automne',
 18: '2024/Automne',
 19: '2024/Automne',
 20: '2024/Automne',
 21: '2024/Automne',
 22: '2024/Automne',
 23: '2024/Automne',
 24: '2024/Automne',
 25: '2024/Automne',
 26: '2024/Automne',
 27: '2024/Automne',
 28: '2024/Automne',
 29: '2024/Automne',
 30: '2024/Automne',
 31: '2024/Automne',
 32: '2024/Automne',
 33: '2024/Automne',
 34: '2024/Automne',
 35: '2024/Automne',
 36: '2024/Automne',
 37: '2024/Automne',
 38: '2024/Automne',
 39: '2024/Automne',
 40: '2024/Automne',
 41: '2024/Automne',
 42: '2024/Hiver',
 43: '2024/Hiver',
 44: '2024/Hiver',
 45: '2024/Hiver',
 46: '2024/Hiver',
 47: '2024/Hiver',
 48: '2024/Hiv

In [53]:
data["folder_path"] = data.index.map(index_to_path)

In [54]:
data

,index,image_name,path,date_time,latitude,longitude,cluster,categories,folder_path
0,0,20240902_150137.jpg,..\photos_final\20240902_150137.jpg,2024:09:02 15:01:38,NaN,NaN,cluster_0,2024_09_02_Randonnée,2024/Automne
1,1,20240902_152654.jpg,..\photos_final\20240902_152654.jpg,2024:09:02 15:26:54,NaN,NaN,cluster_0,2024_09_02_Randonnée,2024/Automne
2,2,20240902_154603.jpg,..\photos_final\20240902_154603.jpg,2024:09:02 15:46:03,NaN,NaN,cluster_0,2024_09_02_Randonnée,2024/Automne
3,3,20240902_154610.jpg,..\photos_final\20240902_154610.jpg,2024:09:02 15:46:10,NaN,NaN,cluster_0,2024_09_02_Randonnée,2024/Automne
4,4,20240902_154612.jpg,..\photos_final\20240902_154612.jpg,2024:09:02 15:46:13,NaN,NaN,cluster_1,Autres/Randonnée,2024/Automne
...,...,...,...,...,...,...,...,...,...
93,93,IMG_20250309_151441.jpg,..\photos_final\IMG_20250309_151441.jpg,2025:03:09 15:14:42,48.408989,-71.053697,cluster_23,2025_03_09_Nourriture,2025/Printemps/Saguenay_CA
94,94,IMG_20250309_200311.jpg,..\photos_final\IMG_20250309_200311.jpg,2025:03:09 20:03:11,NaN,NaN,cluster_24,Autres/Bâtiment,2025/Printemps
95,95,IMG_20250309_200336.jpg,..\photos_final\IMG_20250309_200336.jpg,2025:03:09 20:03:37,48.425122,-71.067358,cluster_25,2025_03_09_Nourriture,2025/Printemps/Saguenay_CA
96,96,IMG_20250309_200346.jpg,..\photos_final\IMG_20250309_200346.jpg,2025:03:09 20:03:46,48.425122,-71.067358,cluster_25,2025_03_09_Nourriture,2025/Printemps/Saguenay_CA
